In [42]:
import numpy as np 
import matplotlib as plt 
import os 
import time
import cv2 
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,models,activations
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input,LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

## Extract keypoints using media pipe hands

In [3]:
#code derived from https://google.github.io/mediapipe/solutions/hands.html
#mediapipe models
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

# detect landmarks/keypoints on hands 
def mp_detect(image, model):
    image.flags.writeable = False 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# draw landmarks on hands
def mp_draw_landmarks(image, results):
    image.flags.writeable = True 
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.get_default_hand_landmarks_style(),
                                      mp_drawing_styles.get_default_hand_connections_style())
            


In [5]:
# testing hand recongition 
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        # capture frame 
        retval, image = cap.read()
        
        # detect landmarks/keypoints 
        image, results = mp_detect(image,hands)
        
        # draw landmarks/keypoints 
        mp_draw_landmarks(image,results)
        
        # visualise
        cv2.imshow('OpenCV Feed', image)
        # quit with q press 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


In [4]:
# extract keypoints 
# 21 keypoints for each hand on the screen 
def extract_keypoints(results):    
    if results.multi_hand_landmarks:
        h1 = np.array([[lm.x, lm.y, lm.z] 
                       for lm in results.multi_hand_landmarks[0].landmark]).flatten()
        # check for second hand 
        if len(results.multi_hand_landmarks) == 2:
            h2 = np.array([[lm.x, lm.y, lm.z] 
                   for lm in results.multi_hand_landmarks[1].landmark]).flatten()
        else:
            h2 = np.zeros(21*3)
        return np.concatenate([h1,h2])
    else: 
        return np.zeros(42*3)
    

## Collect Training and Testing Data 

In [5]:
gestures = np.array(['play','pause'])
no_datapoints = 30
no_frames = 30
DATA_PATH = os.path.join(os.getcwd(),"Gestures_Data")

In [6]:
#create folders for each datapoint collected from videos 
for gesture in gestures:
    for datapoint in range (1,no_datapoints+1):
        try:
            os.makedirs(os.path.join(DATA_PATH,gesture,str(datapoint)))
        except:
            pass
    

In [7]:
# collect and save video data of the gestures as np arrays, 30 frames per video
font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.5) as hands:
    for gesture in gestures:
        for datapoint in range(1,no_datapoints+1): 
            for frame_num in range(no_frames):
                
                # capture frame 
                retval, image = cap.read()

                # detect landmarks/keypoints 
                image, results = mp_detect(image,hands)
                
                # draw landmarks/keypoints
                mp_draw_landmarks(image,results)
                
                # show text at the start and end of the recording 
                # start of recording 
                if frame_num == 0:
                    cv2.putText(image,'START',(250,260),font,2,(0,255,0),2, cv2.LINE_AA)
                    cv2.putText(image,'Gesture: {} || Video: {}'.format(gesture,str(datapoint)),(20, 20),
                               font,0.5,(0,255,0),1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    # wait 2 second 
                    cv2.waitKey(500)
                # end of recording 
                elif frame_num == no_frames-1:
                    cv2.putText(image,'END',(250,260),font,2,(0,255,0),2, cv2.LINE_AA)
                    cv2.putText(image,'Gesture: {} || Video: {}'.format(gesture,str(datapoint)),(20, 20),
                               font,0.5,(0,255,0),1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    # wait 2 seconds
                    cv2.waitKey(2000)

                else:
                    cv2.putText(image,'Gesture: {} || Video: {}'.format(gesture,str(datapoint)),(20, 20),
                               font,0.5,(0,255,0),1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    
                # extract and save keypoints for each frame
                keypoints = extract_keypoints(results)
                path = os.path.join(DATA_PATH,gesture,str(datapoint),str(frame_num))
                np.save(path,keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

## Preprocess data 

In [9]:
def load_data(gesture,datapoint):
    label = gesture
    data = []
    for frame_num in range(no_frames):
        frame = np.load(os.path.join(DATA_PATH, gesture, str(datapoint),"{}.npy".format(frame_num)))
        data.append(frame)
    return data, label       


In [ ]:
labels_map = {label:num for num, label in enumerate(gestures)}
datapoints, labels = [], []
for gesture in gestures:
    for datapoint in range(1,no_datapoints+1):
        data, label = load_data(gesture,datapoint)
        datapoints.append(data)
        labels.append(labels_map[gesture])

In [22]:
X = np.array(datapoints)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)


## Create and Train Basic LSTM Model 

In [74]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [75]:
inputs = Input((30,126))
lstm = LSTM(64,return_sequences=True, activation = 'relu',input_shape=(30,126))(inputs)
lstm2 = LSTM(128, activation = 'relu')(lstm)
de = Dense(128, activation='relu')(lstm2)
de2 = Dense(64, activation='relu')(de)
outputs = Dense(2, activation='softmax')(de2)
model = models.Model(inputs,outputs)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 30, 126)]         0         
_________________________________________________________________
lstm_67 (LSTM)               (None, 30, 64)            48896     
_________________________________________________________________
lstm_68 (LSTM)               (None, 128)               98816     
_________________________________________________________________
dense_42 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_43 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 130       
Total params: 172,610
Trainable params: 172,610
Non-trainable params: 0
_____________________________________________________

In [76]:
model.fit(X_train, y_train, epochs=250, callbacks=[tb_callback])

Epoch 1/250
2/2 [==============================] - 3s 1s/step - loss: 0.6862 - categorical_accuracy: 0.5556
Epoch 2/250
2/2 [==============================] - 1s 311ms/step - loss: 0.6589 - categorical_accuracy: 0.7778
Epoch 3/250
2/2 [==============================] - 1s 260ms/step - loss: 0.6193 - categorical_accuracy: 0.8704
Epoch 4/250
2/2 [==============================] - 1s 304ms/step - loss: 0.5567 - categorical_accuracy: 0.8333
Epoch 5/250
2/2 [==============================] - 1s 305ms/step - loss: 0.4435 - categorical_accuracy: 0.8519
Epoch 6/250
2/2 [==============================] - 1s 304ms/step - loss: 0.2789 - categorical_accuracy: 0.9074
Epoch 7/250
2/2 [==============================] - 1s 285ms/step - loss: 0.1817 - categorical_accuracy: 0.9259
Epoch 8/250
2/2 [==============================] - 1s 271ms/step - loss: 0.1479 - categorical_accuracy: 0.9630
Epoch 9/250
2/2 [==============================] - 1s 304ms/step - loss: 0.1503 - categorical_accuracy: 0.9259
Epoc

2/2 [==============================] - 1s 265ms/step - loss: 0.1102 - categorical_accuracy: 0.9815
Epoch 75/250
2/2 [==============================] - 1s 259ms/step - loss: 0.1017 - categorical_accuracy: 0.9815
Epoch 76/250
2/2 [==============================] - 1s 259ms/step - loss: 0.1009 - categorical_accuracy: 0.9815
Epoch 77/250
2/2 [==============================] - 1s 266ms/step - loss: 0.1054 - categorical_accuracy: 0.9815
Epoch 78/250
2/2 [==============================] - 1s 267ms/step - loss: 0.1139 - categorical_accuracy: 0.9815
Epoch 79/250
2/2 [==============================] - 1s 269ms/step - loss: 0.1136 - categorical_accuracy: 0.9815
Epoch 80/250
2/2 [==============================] - 1s 262ms/step - loss: 0.1048 - categorical_accuracy: 0.9815
Epoch 81/250
2/2 [==============================] - 1s 258ms/step - loss: 0.0945 - categorical_accuracy: 0.9815
Epoch 82/250
2/2 [==============================] - 1s 268ms/step - loss: 0.0857 - categorical_accuracy: 0.9815
Epoch

2/2 [==============================] - 0s 218ms/step - loss: 0.1153 - categorical_accuracy: 0.9630
Epoch 147/250
2/2 [==============================] - 0s 246ms/step - loss: 0.2933 - categorical_accuracy: 0.8889
Epoch 148/250
2/2 [==============================] - 1s 273ms/step - loss: 0.4037 - categorical_accuracy: 0.8148
Epoch 149/250
2/2 [==============================] - 1s 299ms/step - loss: 0.3011 - categorical_accuracy: 0.8889
Epoch 150/250
2/2 [==============================] - 1s 253ms/step - loss: 0.2266 - categorical_accuracy: 0.8889
Epoch 151/250
2/2 [==============================] - 1s 260ms/step - loss: 0.1717 - categorical_accuracy: 0.9444
Epoch 152/250
2/2 [==============================] - 1s 261ms/step - loss: 0.1604 - categorical_accuracy: 0.9630
Epoch 153/250
2/2 [==============================] - 1s 259ms/step - loss: 0.1593 - categorical_accuracy: 0.9815
Epoch 154/250
2/2 [==============================] - 1s 270ms/step - loss: 0.1560 - categorical_accuracy: 0.98

2/2 [==============================] - 1s 307ms/step - loss: 0.0555 - categorical_accuracy: 0.9815
Epoch 219/250
2/2 [==============================] - 1s 303ms/step - loss: 0.2077 - categorical_accuracy: 0.9815
Epoch 220/250
2/2 [==============================] - 1s 307ms/step - loss: 0.1293 - categorical_accuracy: 0.9444
Epoch 221/250
2/2 [==============================] - 1s 261ms/step - loss: 0.2180 - categorical_accuracy: 0.9259
Epoch 222/250
2/2 [==============================] - 1s 269ms/step - loss: 0.2957 - categorical_accuracy: 0.8333
Epoch 223/250
2/2 [==============================] - 1s 299ms/step - loss: 0.2925 - categorical_accuracy: 0.8519
Epoch 224/250
2/2 [==============================] - 1s 303ms/step - loss: 0.2407 - categorical_accuracy: 0.9259
Epoch 225/250
2/2 [==============================] - 1s 272ms/step - loss: 0.1909 - categorical_accuracy: 0.9444
Epoch 226/250
2/2 [==============================] - 1s 304ms/step - loss: 0.1398 - categorical_accuracy: 0.96

In [ ]:
model.save('action.h5')